In [6]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import itertools
import torch as th
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback
import matplotlib

# =======================================
# Ввод начального состояния
# =======================================
def get_initial_state():
    while True:
        try:
            inp = input("Введите 5 целых неотрицательных чисел (объёмы в мл): ")
            volumes = list(map(int, inp.split()))
            if len(volumes) != 5:
                print("Нужно ровно 5 чисел!")
                continue
            if any(v < 0 for v in volumes):
                print("Объёмы не могут быть отрицательными!")
                continue
            if sum(volumes) == 0:
                print("Общий объём не может быть нулевым!")
                continue
            arr = np.array(volumes, dtype=np.int32)
            if arr.max() > 500:
                print("Объём в одном стакане не должен превышать 500 мл.")
                continue
            if arr.sum() > 2500:
                print("Суммарный объём не должен превышать 2500 мл.")
                continue
            return arr
        except ValueError:
            print("Некорректный ввод. Пример: 200 0 0 0 0")

# =======================================
# Выбор шага переливания
# =======================================
def get_volume_step(max_vol):
    if max_vol <= 50:
        return 1
    elif max_vol <= 100:
        return 2
    elif max_vol <= 200:
        return 5
    elif max_vol <= 500:
        return 10
    else:
        return 10

# =======================================
# Среда
# =======================================
class CompoteEnv(gym.Env):
    def __init__(self, initial_state: np.ndarray, max_steps=100, volume_step=10, max_volume=500):
        super().__init__()
        self.n_glasses = 5
        self._initial_state_frozen = initial_state.copy()
        self.total_volume = int(self._initial_state_frozen.sum())
        self.max_steps = max_steps
        self.volume_step = volume_step
        self.max_volume = max_volume

        # Целевое распределение (как Python int, не np.int64)
        base = self.total_volume // self.n_glasses
        rem = self.total_volume % self.n_glasses
        self.target = [int(base + 1)] * rem + [int(base)] * (self.n_glasses - rem)

        # Наблюдение
        self.observation_space = spaces.Box(
            low=0, high=self.max_volume, shape=(5,), dtype=np.float32
        )

        # Максимальный объём для переливания — только тот, что был в начальном состоянии
        self.max_possible_pour = int(self._initial_state_frozen.max())

        # Генерация действий: (источник, (цели,), объём)
        self._actions = []
        for src in range(5):
            others = [i for i in range(5) if i != src]
            for r in range(1, len(others) + 1):
                for subset in itertools.combinations(others, r):
                    vol = self.volume_step
                    while vol <= self.max_possible_pour:
                        self._actions.append((src, subset, vol))
                        vol += self.volume_step
        self.action_space = spaces.Discrete(len(self._actions))

        self.reset()

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.state = self._initial_state_frozen.copy().astype(np.float32)
        # Инициализируем prev_deviation текущим отклонением
        initial_dev = np.abs(np.sort(self._initial_state_frozen) - np.sort(self.target)).sum()
        self.prev_deviation = float(initial_dev)
        self.steps = 0
        return self.state, {}

    def _is_balanced(self, state):
        return sorted([int(x) for x in state]) == sorted(self.target)

    def step(self, action_idx):
        self.steps += 1
        action = self._actions[action_idx]

        src, targets, volume_to_pour = action
        available = self.state[src]

        if volume_to_pour > available or volume_to_pour <= 0:
            done = self._is_balanced(self.state)
            truncated = self.steps >= self.max_steps
            reward = -100.0
            return self.state.copy(), reward, done, truncated, {}

        self.state[src] -= volume_to_pour
        n = len(targets)
        per_target = volume_to_pour // n
        remainder = int(volume_to_pour - per_target * n)

        for i, t in enumerate(targets):
            self.state[t] += per_target
            if i < remainder:
                self.state[t] += 1

        self.state = np.clip(self.state, 0, self.max_volume)

        done = self._is_balanced(self.state)
        truncated = self.steps >= self.max_steps

        # === КЛЮЧЕВОЕ ИЗМЕНЕНИЕ: НАГРАДА ===
        curr_dev = np.abs(np.sort(self.state) - np.sort(self.target)).sum()
        if done:
            reward = 200.0
        else:
            improvement = self.prev_deviation - curr_dev
            reward = improvement * 0.5 - 1.0
        self.prev_deviation = curr_dev
        return self.state.copy(), reward, done, truncated, {}

# =======================================
# Callback для графика
# =======================================
class RewardLoggerCallback(BaseCallback):
    def __init__(self, eval_env, eval_freq=1000, verbose=0):
        super().__init__(verbose)
        self.eval_env = eval_env
        self.eval_freq = eval_freq
        self.rewards = []

    def _on_step(self) -> bool:
        if self.num_timesteps % self.eval_freq == 0:
            obs, _ = self.eval_env.reset()
            episode_reward = 0.0
            steps = 0
            while steps < 100:
                action, _ = self.model.predict(obs, deterministic=False)
                obs, r, d, tr, _ = self.eval_env.step(action)
                episode_reward += r
                steps += 1
                if d or tr:
                    break
            self.rewards.append(episode_reward)
        return True

# =======================================
# Основная функция
# =======================================
def main():
    initial = get_initial_state()
    total_vol = initial.sum()
    max_in_st = initial.max()
    volume_step = get_volume_step(max_in_st)

    print(f"\nНачальное состояние: {initial}")
    print(f"Общий объём: {total_vol} мл")
    print(f"Макс. в стакане: {max_in_st} мл → шаг переливания: {volume_step} мл")

    base = total_vol // 5
    rem = total_vol % 5
    target = [int(base + 1)] * rem + [int(base)] * (5 - rem)
    print(f"Целевое распределение: {sorted(target, reverse=True)}")

    # Создаём две НЕЗАВИСИМЫЕ среды
    env = CompoteEnv(initial, max_steps=100, volume_step=volume_step, max_volume=500)
    eval_env = CompoteEnv(initial, max_steps=100, volume_step=volume_step, max_volume=500)

    print("\n🔍 Проверка среды...")
    check_env(env, warn=True)

    print("\n🧠 Обучение DQN-агента...")
    device = "cpu"

    n_actions = env.action_space.n
    print(f"Число действий: {n_actions}")

    model = DQN(
        "MlpPolicy",
        env,
        buffer_size=min(20000, n_actions * 10),
        learning_starts=3000,
        batch_size=64,
        gamma=0.99,
        exploration_initial_eps=1.0,
        exploration_final_eps=0.001,
        exploration_fraction=0.3,
        train_freq=4,
        gradient_steps=1,
        target_update_interval=1000,
        verbose=0,
        device=device
    )

    total_timesteps = min(150000, n_actions * 200)
    print(f"Общее число шагов обучения: {total_timesteps}")

    reward_logger = RewardLoggerCallback(eval_env, eval_freq=1000)
    model.learn(total_timesteps=total_timesteps, callback=reward_logger, progress_bar=True)

    # === PLOTLY (offline) ===
    rewards = reward_logger.rewards
    if rewards:
        clean_rewards = [float(r) for r in rewards if np.isfinite(r)]
        if clean_rewards:
            import plotly.graph_objects as go
            x_vals = list(range(1000, 1000 * len(clean_rewards) + 1, 1000))
            fig = go.Figure(data=go.Scatter(x=x_vals, y=clean_rewards, mode='lines+markers'))
            fig.update_layout(
                title="Награда агента",
                xaxis_title="Шаги обучения",
                yaxis_title="Суммарная награда"
            )
            fig.write_html("reward_plot.html")
            print("\n✅ Интерактивный график сохранён как 'reward_plot.html'")
        else:
            print("\n⚠️ Нет конечных значений для графика.")

    # === ТЕСТ НА ОТДЕЛЬНОЙ СРЕДЕ ===
    print("\n🧪 Тест обученного агента:")
    obs, _ = eval_env.reset()  # ← ВАЖНО: eval_env, не env!
    print(f"Начальное состояние: {obs.astype(int)} мл")
    done, truncated = False, False
    step = 0
    while not (done or truncated) and step < 100:
        action, _ = model.predict(obs, deterministic=False)
        obs, reward, done, truncated, _ = eval_env.step(action)  # ← eval_env!
        action_desc = eval_env._actions[action]
        step += 1
        src, tgts, vol = action_desc
        print(f"Шаг {step}: из {src} → в {tgts}, объём {vol} мл → {obs.astype(int)} мл")
        if done:
            print("✅ Цель достигнута!")
        elif truncated:
            print("⏱️ Лимит шагов достигнут.")

    final_state = obs.astype(int)
    print(f"\nФинальное состояние: {final_state} мл")
    print(f"Цель: {sorted(target)} мл")
    print(f"Сбалансировано: {sorted(final_state.tolist()) == sorted(target)}")

if __name__ == "__main__":
    main()

Введите 5 целых неотрицательных чисел (объёмы в мл):  10 10 10 10 15


Output()


Начальное состояние: [10 10 10 10 15]
Общий объём: 55 мл
Макс. в стакане: 15 мл → шаг переливания: 1 мл
Целевое распределение: [11, 11, 11, 11, 11]

🔍 Проверка среды...

🧠 Обучение DQN-агента...
Число действий: 1125
Общее число шагов обучения: 150000



✅ Интерактивный график сохранён как 'reward_plot.html'

🧪 Тест обученного агента:
Начальное состояние: [10 10 10 10 15] мл
Шаг 1: из 4 → в (1, 2, 3), объём 4 мл → [10 12 11 11 11] мл
Шаг 2: из 1 → в (0, 3, 4), объём 1 мл → [11 11 11 11 11] мл
✅ Цель достигнута!

Финальное состояние: [11 11 11 11 11] мл
Цель: [11, 11, 11, 11, 11] мл
Сбалансировано: True
